In [1]:
from urllib import request
import json
import numpy as np
import pandas as pd 

In [2]:
def get_list(keyword="country"):
    response = request.urlopen("https://nomadlist.com/api/v2/list/{}".format(keyword))
    resp_string = response.read().decode("utf-8")
    resp_dict = json.loads(resp_string)
    return resp_dict['result']

In [3]:
place_list = get_list('places')

In [4]:
def transform_cost_dict(dict_cost):
    for k,v in dict_cost.items():
        dict_cost[k] = v['USD']
    return dict_cost 

places = []
for place in place_list:
    new_dict = place['scores']
    x = transform_cost_dict(place['cost'])
    new_dict.update(x)
    new_dict["city_name"] = place["media"]["image"]["500"].split("/")[-1].split("500px")[0].strip("-")
    places.append(new_dict)

In [5]:
newdf = pd.DataFrame(places)

In [6]:
def get_corect_country(country, countries):
    for c in countries:
        if country in c:
            return c

In [7]:
newdf['country'] = newdf['city_name'].apply(lambda x: x.split("-")[-1])
country_list = get_list("countries")
newdf['country_slug'] = newdf['country'].apply(lambda x: get_corect_country(x, country_list))
newdf['country_name'] = newdf['country_slug'].apply(lambda x: x.replace('-', ' '))

In [8]:
newdf.head()

,aircon,city_name,expat,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,leisure,lgbt_friendly,...,nomad_score,peace_score,places_to_work,press_freedom_index,racism,safety,shortTerm,country,country_slug,country_name
0,0.6,chiang-mai-thailand,620,0.8,78.8,0.6,0.2,0.6,0.62,0.6,...,1.00,0.4,1.0,44.53,0.40,0.80,599,thailand,thailand,thailand
1,1.0,bangkok-thailand,968,0.8,78.8,1.0,0.2,1.0,0.82,0.8,...,0.96,0.4,1.0,44.53,0.42,0.77,1082,thailand,thailand,thailand
2,1.0,taipei-taiwan,1498,1.0,NaN,1.0,0.6,0.8,1.00,0.8,...,0.90,NaN,1.0,24.37,0.00,1.00,1401,taiwan,taiwan,taiwan
3,1.0,melbourne-australia,2703,1.0,22.5,0.2,0.8,0.8,0.80,0.8,...,0.89,1.0,1.0,17.84,0.80,0.82,2741,australia,australia,australia
4,0.6,canggu-bali-indonesia,1283,0.8,74.9,0.6,0.2,1.0,0.90,0.6,...,0.89,0.6,0.8,41.72,0.80,0.60,951,indonesia,indonesia,indonesia


In [9]:
grouped = newdf.groupby("country_name").mean()

In [10]:
grouped.columns

Index(['aircon', 'expat', 'female_friendly', 'fragile_states_index',
       'free_wifi_available', 'freedom_score', 'friendly_to_foreigners',
       'leisure', 'lgbt_friendly', 'life_score', 'local', 'longTerm',
       'nightlife', 'nomad', 'nomadScore', 'nomad_score', 'peace_score',
       'places_to_work', 'press_freedom_index', 'racism', 'safety',
       'shortTerm'],
      dtype='object')

In [11]:
grouped.to_json('data.json')

In [20]:
grouped.sample(10)

,aircon,expat,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,leisure,lgbt_friendly,life_score,...,nightlife,nomad,nomadScore,nomad_score,peace_score,places_to_work,press_freedom_index,racism,safety,shortTerm
country_name,,,,,,,,,,,,,,,,,,,,,
iraq,0.300000,2383.000000,0.200000,104.7,0.300000,0.200000,0.500000,0.300000,0.200000,0.115000,...,0.200000,8364.500000,0.115000,0.115000,0.200000,0.300000,54.350000,0.400000,0.200000,8364.500000
south africa,0.604000,1248.200000,0.560000,69.9,0.560000,0.600000,0.760000,0.680000,0.680000,0.640000,...,0.560000,2376.400000,0.580000,0.580000,0.600000,0.640000,21.920000,0.600000,0.404000,2376.400000
malta,0.400000,1391.000000,0.800000,39.6,0.600000,0.600000,0.800000,0.800000,0.800000,0.870000,...,0.600000,2155.000000,0.730000,0.730000,0.800000,0.400000,23.840000,0.000000,0.800000,2155.000000
thailand,0.640000,866.400000,0.700000,78.8,0.440000,0.200000,0.840000,0.646000,0.760000,0.753000,...,0.685000,894.000000,0.752000,0.752000,0.400000,0.580000,44.530000,0.404000,0.733000,894.000000
oman,1.000000,1876.000000,0.600000,51.6,0.400000,0.200000,0.600000,0.800000,0.400000,0.760000,...,0.400000,3712.000000,0.650000,0.650000,0.800000,0.600000,40.430000,0.000000,0.800000,3712.000000
czech republic,0.466667,1579.666667,0.733333,60.8,0.666667,0.666667,0.866667,0.866667,0.466667,0.790000,...,0.866667,1144.666667,0.716667,0.716667,0.666667,0.783333,24.153333,0.673333,0.600000,1144.666667
hungary,0.800000,1023.000000,1.000000,52.7,0.400000,0.600000,0.600000,0.800000,0.270000,0.890000,...,1.000000,1000.000000,0.850000,0.850000,0.800000,1.000000,28.170000,0.400000,0.600000,1000.000000
turkey,0.622222,736.888889,0.530000,77.3,0.527778,0.200000,0.685556,0.621111,0.331111,0.674444,...,0.577778,1608.222222,0.585556,0.585556,0.400000,0.466667,50.760000,0.486667,0.703333,1608.222222
myanmar,0.800000,912.000000,0.800000,96.3,0.800000,NaN,0.800000,1.000000,0.400000,0.540000,...,0.400000,751.000000,0.540000,0.540000,0.200000,0.400000,NaN,0.000000,0.600000,751.000000


In [27]:
df = pd.read_csv('codes.csv')

In [28]:
df = df[['Content', 'Value 3']]
df['country'] = df['Content'].apply(lambda x: x.lower())
df['code'] = df['Value 3']

In [29]:
df = df.set_index('country')

In [30]:
g1 = grouped.join(df['code'])

In [57]:
g1['code'][g1.code.isnull()].keys()

Index(['bosnia herzegovina', 'congo', 'hong kong', 'ivory coast', 'kurdistan',
       'laos', 'macau', 'macedonia', 'north korea', 'palestine', 'russia',
       'syria', 'taiwan', 'tanzania', 'united states', 'venezuela', 'vietnam'],
      dtype='object', name='country_name')

In [33]:
grouped.shape

(131, 22)

In [58]:
g1.set_value('iran', 'code', 'IRN')
g1.set_value('bosnia herzegovina', 'code', 'BIH')
g1.set_value('congo', 'code', 'COD')
g1.set_value('hong kong', 'code', 'HKG')
g1.set_value('ivory coast', 'code', 'CIV')
g1.set_value('kurdistan', 'code', 'KUR')
g1.set_value('laos', 'code', 'LAO')
g1.set_value('macau', 'code', 'MAC')
g1.set_value('macedonia', 'code', 'MKD')
g1.set_value('north korea', 'code', 'PRK')
g1.set_value('palestine', 'code', 'PSE')
g1.set_value('russia', 'code', 'RUS')
g1.set_value('syria', 'code', 'SYR')
g1.set_value('taiwan', 'code', 'TWN')
g1.set_value('tanzania', 'code', 'TZA')
g1.set_value('united states', 'code', 'USA')
g1.set_value('venezuela', 'code', 'VEN')
g1.set_value('vietnam', 'code', 'VNM')

,aircon,expat,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,leisure,lgbt_friendly,life_score,...,nomad,nomadScore,nomad_score,peace_score,places_to_work,press_freedom_index,racism,safety,shortTerm,code
country_name,,,,,,,,,,,,,,,,,,,,,
albania,0.600000,778.000000,0.600000,61.2,0.600000,0.600000,0.800000,0.400000,0.600000,0.670000,...,1234.000000,0.630000,0.630000,0.600000,0.400000,29.920000,0.000000e+00,0.600000,1234.000000,ALB
algeria,0.600000,773.000000,0.200000,78.3,0.400000,0.200000,0.400000,0.600000,0.200000,0.650000,...,2095.000000,0.610000,0.610000,0.400000,0.200000,41.690000,4.000000e-01,0.250000,2095.000000,DZA
angola,0.500000,3749.000000,0.500000,90.5,0.500000,0.400000,0.800000,0.600000,0.400000,0.370000,...,6559.000000,0.355000,0.355000,0.200000,0.600000,39.890000,2.000000e-01,0.300000,6559.000000,AGO
argentina,0.640000,870.500000,0.566667,48.4,0.496667,0.600000,0.770000,0.766667,0.833333,0.773333,...,2065.666667,0.708333,0.708333,0.800000,0.633333,25.090000,7.300000e-01,0.460000,2065.666667,ARG
armenia,0.640000,792.000000,0.820000,69.6,0.640000,0.600000,0.640000,0.820000,0.420000,0.690000,...,1167.000000,0.600000,0.600000,0.600000,0.800000,28.790000,0.000000e+00,0.820000,1167.000000,ARM
aruba,0.600000,1497.000000,0.400000,NaN,0.600000,NaN,0.800000,0.800000,0.400000,0.860000,...,3807.000000,0.740000,0.740000,NaN,0.400000,NaN,6.000000e-01,1.000000,3807.000000,ABW
australia,0.822222,2951.777778,0.911111,22.5,0.600000,0.800000,0.888889,0.842222,0.844444,0.893333,...,2864.333333,0.774444,0.774444,1.000000,0.733333,17.840000,7.777778e-01,0.846667,2864.333333,AUS
austria,0.545000,1672.500000,0.900000,27.5,0.700000,0.800000,0.800000,0.605000,0.655000,0.827500,...,2802.000000,0.635000,0.635000,1.000000,0.600000,13.180000,6.000000e-01,0.850000,2802.000000,AUT
azerbaijan,0.600000,722.000000,0.400000,76.3,0.400000,0.200000,0.800000,0.800000,0.400000,0.680000,...,2726.000000,0.540000,0.540000,0.400000,0.400000,57.890000,0.000000e+00,0.600000,2726.000000,AZE


In [62]:
g1 = g1.set_index('code')

In [63]:
g1.to_json('data.json')

In [65]:
g1.shape

(131, 22)

In [68]:
grouped

,aircon,expat,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,leisure,lgbt_friendly,life_score,...,nightlife,nomad,nomadScore,nomad_score,peace_score,places_to_work,press_freedom_index,racism,safety,shortTerm
country_name,,,,,,,,,,,,,,,,,,,,,
albania,0.600000,778.000000,0.600000,61.2,0.600000,0.600000,0.800000,0.400000,0.600000,0.670000,...,0.800000,1234.000000,0.630000,0.630000,0.600000,0.400000,29.920000,0.000000e+00,0.600000,1234.000000
algeria,0.600000,773.000000,0.200000,78.3,0.400000,0.200000,0.400000,0.600000,0.200000,0.650000,...,0.400000,2095.000000,0.610000,0.610000,0.400000,0.200000,41.690000,4.000000e-01,0.250000,2095.000000
angola,0.500000,3749.000000,0.500000,90.5,0.500000,0.400000,0.800000,0.600000,0.400000,0.370000,...,0.600000,6559.000000,0.355000,0.355000,0.200000,0.600000,39.890000,2.000000e-01,0.300000,6559.000000
argentina,0.640000,870.500000,0.566667,48.4,0.496667,0.600000,0.770000,0.766667,0.833333,0.773333,...,0.766667,2065.666667,0.708333,0.708333,0.800000,0.633333,25.090000,7.300000e-01,0.460000,2065.666667
armenia,0.640000,792.000000,0.820000,69.6,0.640000,0.600000,0.640000,0.820000,0.420000,0.690000,...,0.640000,1167.000000,0.600000,0.600000,0.600000,0.800000,28.790000,0.000000e+00,0.820000,1167.000000
aruba,0.600000,1497.000000,0.400000,NaN,0.600000,NaN,0.800000,0.800000,0.400000,0.860000,...,0.400000,3807.000000,0.740000,0.740000,NaN,0.400000,NaN,6.000000e-01,1.000000,3807.000000
australia,0.822222,2951.777778,0.911111,22.5,0.600000,0.800000,0.888889,0.842222,0.844444,0.893333,...,0.666667,2864.333333,0.774444,0.774444,1.000000,0.733333,17.840000,7.777778e-01,0.846667,2864.333333
austria,0.545000,1672.500000,0.900000,27.5,0.700000,0.800000,0.800000,0.605000,0.655000,0.827500,...,0.590000,2802.000000,0.635000,0.635000,1.000000,0.600000,13.180000,6.000000e-01,0.850000,2802.000000
azerbaijan,0.600000,722.000000,0.400000,76.3,0.400000,0.200000,0.800000,0.800000,0.400000,0.680000,...,1.000000,2726.000000,0.540000,0.540000,0.400000,0.400000,57.890000,0.000000e+00,0.600000,2726.000000
